<a href="https://colab.research.google.com/github/pietrzakkuba/natural-language-processing-labs/blob/master/lab_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Komputerowe wspomaganie tłumaczenia

# Zajęcia 1 - podstawowe techniki wspomagania tłumaczenia

Podstawowy scenariusz - tłumacz otrzymuje **dokument** będący ciągiem **segmentów** w **języku źródłowym** . Zadaniem tłumacza jest przetłumaczenie wszystkich segmentów na **język docelowy**. Oznacza to, że został wykonany już szereg operacji technicznych, a w tym:
- przesłanie pliku do tłumaczenia
- ekstrakcja tekstu z dokumentu
- podział tekstu na segmenty.


Tłumacz tłumaczy jeden segment na raz. Widzi kilka poprzednich oraz kilka następnych segmentów, jednak jego uwaga powinna skupiać się na aktywnym, aktualnie tłumaczonym segmencie. Nie oznacza to jednak, że tłumacz jest pozbawiony pomocy. Ma możliwość korzystania z różnorakich technik **komputerowego wspomagania tłumaczenia**.

# Pamięć tłumaczeń

Najbardziej podstawową techniką wspomagania tłumaczenia jest **pamięć tłumaczeń**. Pamięć tłumaczeń stanowi zbiór wcześniej przetłumaczonych zdań:

In [ ]:
translation_memory = [('Wciśnij przycisk Enter', 'Press the ENTER button'), 
                      ('Sprawdź ustawienia sieciowe', 'Check the network settings')]

In [ ]:
def tm_lookup(sentence):
    return [entry[1] for entry in translation_memory if entry[0] == sentence]

In [ ]:
tm_lookup('Wciśnij przycisk Enter')

['Press the ENTER button']

W takim przypadku tłumaczenie z pamięci jest najczęściej automatycznie wstawiane jako tłumaczenie segmentu źródłowego i tłumacz nie musi się już nim martwić (co więcej, nie dostaje za takie tłumaczenie zapłaty...)

Tłumacz pracuje dalej i każde wykonane przez niego tłumaczenie trafia do pamięci tłumaczeń (oczywiście dzieje się to automatycznie):

In [ ]:
translation_memory.append(('Drukarka jest wyłączona', 'The printer is switched off'))
translation_memory.append(('Wymagane ponowne uruchomienie komputera', 'System restart required'))
translation_memory.append(('Wciśnij przycisk Enter', 'Press the ENTER key'))

Z całą pewnością tłumacz stara się przełożyć jakąś komputerową instrukcję. Po wielu godzinach pracy trafia na znane sobie zdanie "Wciśnij przycisk Enter". Następuje przeszukanie pamięci tłumaczeń:

In [ ]:
tm_lookup('Wciśnij przycisk Enter')

['Press the ENTER button', 'Press the ENTER key']

Jak widać, mamy w pamięci dwa tłumaczenia tego zdania. To tłumacz musi zadecydować, które tłumaczenie wybrać. O metodach wspomagania decyzji tłumacza w takich przypadkach będziemy jeszcze mówić.

Przeanalizujmy teraz następujący przypadek: do tłumaczenia trafia zdanie: "Wciśnij przycisk ENTER":

In [ ]:
tm_lookup('Wciśnij przycisk ENTER')

[]

Tutaj tłumacz nie ma szczęścia - restrykcyjny mechanizm wyszukiwania nie podaje mu podpowiedzi. Możemy jednak sprawić, żeby przeszukiwanie pamięci tłumaczeń nie brało pod uwagę wielkości liter.

### Ćwiczenie 1: zmodyfikuj funkcję tm_lookup w taki sposób, aby nie brała pod uwagę wielkości liter. Przetestuj zmodyfikowaną funkcję.

In [ ]:
def tm_lookup(sentence):
    return [entry[1] for entry in translation_memory if entry[0].lower() == sentence.lower()]

tm_lookup('Wciśnij przycisk ENTER')

['Press the ENTER button', 'Press the ENTER key']

Nasz tłumacz powinien być teraz zadowolony. Jednak w dalszej części dokumentu pojawiło się inne podobne zdanie, dla którego nie było tłumaczenia:

In [ ]:
tm_lookup('Wciśnij przycisk [ENTER]')

[]

### Ćwiczenie 2: zmodyfikuj funkcję tm_lookup w taki sposób, aby nie brała pod uwagę znaków interpunkcyjnych. Rada - zdefiniuj funkcję sentence_similar.

In [ ]:
from string import punctuation

def sentence_similar(sentence):
    new_sentence = sentence.lower()
    for char in punctuation:
        if char in sentence:
            new_sentence = new_sentence.replace(char, '')
    return new_sentence

def tm_lookup(sentence):
    return [entry[1] for entry in translation_memory if sentence_similar(entry[0]) == sentence_similar(sentence)]

tm_lookup('Wciśnij przycisk [ENTER]')

['Press the ENTER button', 'Press the ENTER key']

Praca tłumacza oczywiście trwa nadal. Spróbujmy teraz zaradzić następującej sytuacji - tłumacz otrzymuje do tłumaczenia następujące zdanie:

In [ ]:
tm_lookup('Wymagane ponowne uruchomienie maszyny')

[]

Znów nie otrzymuje od nas podpowiedzi i znów nic w tym dziwnego - w pamięci tłumaczeń nie znajduje się takie zdanie. Jest jednak zdanie podobne, różniące się jednym słowem.

### Ćwiczenie 3: zmodyfikuj funkcję tm_lookup w taki sposób, aby zwracała podpowiedzi także wtedy, gdy zdania różnią się tylko jednym słowem.

In [ ]:
def have_max_one_different_word(sen1, sen2):
    diffs = 0
    sen1_words = sentence_similar(sen1).split()
    sen2_words = sentence_similar(sen2).split()
    for (word1, word2) in zip(sen1_words, sen2_words):
        if word1 != word2:
            diffs += 1
            if diffs > 1:
                return False
    return True



def tm_lookup(sentence):
    return [entry[1] for entry in translation_memory if have_max_one_different_word(entry[0], sentence)]

tm_lookup('Wymagane ponowne uruchomienie maszyny')

['System restart required']

# Słownik kontekstowy / glosariusz

Inną podstawową pomocą dla naszego tłumacza jest słownik, zwany także glosariuszem. Stanowi on zbiór terminów w języku źródłowym wraz z ich tłumaczeniami na język docelowy:

In [ ]:
glossary = [('komputer', 'computer'), ('przycisk', 'button'), ('drukarka', 'printer')]

Podczas tłumaczenia, glosariusz jest przeszukiwany w celu znalezienia aktualnie tłumaczonych słów.

In [ ]:
def glossary_lookup(sentence):
    sentence_words = sentence.split()
    return [entry for entry in glossary if entry[0] in sentence_words]

In [ ]:
glossary_lookup('Każda drukarka posiada przycisk wznowienia drukowania')

[('przycisk', 'button'), ('drukarka', 'printer')]

### Ćwiczenie 4: (tym razem teoretyczne) Jaka jest złożoność obliczeniowa czasowa przedstawionego powyżej algorytmu względem liczby haseł w słowniku (n) oraz liczby słów w zdaniu do tłumaczenia (m)?

Odpowiedź: 
Złożonośc wynosi n * m. 
Uzasadnienie: 
dla każdego hasła w glosariuszu (czyli n razy) przeszukujumy całe zdanie dokonując m porównań.  

### Ćwiczenie 5: zmodyfikuj funkcję glossary_lookup w taki sposób, aby nie brała pod uwagę wielkości liter.

In [ ]:
def glossary_lookup(sentence):
    sentence_words = [word.lower() for word in sentence.split()]
    return [entry for entry in glossary if entry[0].lower() in sentence_words]

glossary_lookup('Każda DrukARKA posiada pRZycisk wznowienia drukowania')

[('przycisk', 'button'), ('drukarka', 'printer')]

### Ćwiczenie 6: zmodyfikuj funkcję glossary_lookup w taki sposób, aby jej złożoność obliczeniowa była mniejsza.

In [ ]:
glossary = {
    'komputer': 'computer',
    'przycisk': 'button',
    'drukarka': 'printer'
}
def glossary_lookup(sentence):
    sentence_words = [word.lower() for word in sentence.split()]
    return [(word, glossary[word]) for word in sentence_words if word in glossary]

glossary_lookup('Każda DrukARKA posiada pRZycisk wznowienia drukowania')

# złożoność jest mniejsza, ponieważ wyszukanie danego elementu w słowniku zajmuje O(1)
# a w liście O(n) gdzie n to długość listy

[('drukarka', 'printer'), ('przycisk', 'button')]